<a href="https://colab.research.google.com/github/Nerujan/Colab/blob/main/XLM_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets seqeval torch

# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import classification_report

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
 

In [2]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=9)  # Adjust num_labels as per your dataset

# Define label mapping
label_list = ["O", "B-ORG", "I-ORG", "B-MISC", "I-MISC", "B-COMM", "I-COMM","B-PER", "I-PER"]  # Adjust as needed
label_map = {label: i for i, label in enumerate(label_list)}

# Load your dataset from Google Drive
dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/Final_Tamil.txt"  # Replace with actual path
dataset = load_dataset("text", data_files=dataset_path)

print(label_map)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split: 0 examples [00:00, ? examples/s]

{'O': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-MISC': 3, 'I-MISC': 4, 'B-COMM': 5, 'I-COMM': 6, 'B-PER': 7, 'I-PER': 8}


In [3]:
def parse_text(example):
    words, labels = [], []
    text_data = example.get("text", "").strip()

    if not text_data:
        return {"tokens": [], "ner_tags": []}

    tokens = text_data.split(" ")  # Split based on space
    i = 0
    while i < len(tokens):
        if i + 1 < len(tokens) and tokens[i + 1] in label_map:  # Check if next token is a label
            words.append(tokens[i])  # Append word
            labels.append(label_map[tokens[i + 1]])  # Append corresponding label
            i += 2  # Move to next word-label pair
        else:
            words.append(tokens[i])  # Append word
            labels.append(label_map["O"])  # Default label
            i += 1  # Move to next token

    return {"tokens": words, "ner_tags": labels}

In [4]:
print(parse_text(dataset['train'][208]))

{'tokens': ['மாணவர்கள்'], 'ner_tags': [5]}


In [5]:
# Apply parsing function
dataset = dataset.map(parse_text)

Map:   0%|          | 0/87124 [00:00<?, ? examples/s]

In [6]:
def tokenize_and_align_labels(batch):
    # Tokenize the inputs with truncation and padding
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, padding="max_length", max_length=128, is_split_into_words=True)

    all_labels = []

    # Align the labels with the tokenized inputs
    for i in range(len(batch["tokens"])): # Iterate through each example in the batch using its index
        word_ids = tokenized_inputs.word_ids(batch_index=i) # Get word_ids for the current example
        labels = []
        previous_word_idx = None

        # Ensure each token gets the corresponding label
        for word_idx in word_ids:
            if word_idx is None:
                labels.append(-100)  # Ignore special tokens (e.g., [CLS], [SEP])
            elif word_idx != previous_word_idx:
                labels.append(batch["ner_tags"][i][word_idx])  # Assign the correct label for the word
            else:
                labels.append(batch["ner_tags"][i][word_idx])  # Keep the same label for subword tokens

            previous_word_idx = word_idx

        # Ensure labels list has the same length as tokenized inputs (padding handled)
        labels += [-100] * (len(tokenized_inputs["input_ids"][i]) - len(labels))

        # Add the labels for this example
        all_labels.append(labels)

    # Return the tokenized inputs with labels aligned correctly
    tokenized_inputs["labels"] = all_labels
    # Remove unnecessary attention_mask from the result
    tokenized_inputs.pop("attention_mask", None)

    return tokenized_inputs

# Apply the function to the dataset
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/87124 [00:00<?, ? examples/s]

In [7]:
print(tokenized_datasets)


DatasetDict({
    train: Dataset({
        features: ['text', 'tokens', 'ner_tags', 'input_ids', 'labels'],
        num_rows: 87124
    })
})


In [8]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results", per_device_train_batch_size=8, per_device_eval_batch_size=8,
    num_train_epochs=3, weight_decay=0.01, evaluation_strategy="epoch",
    save_strategy="epoch", logging_dir="./logs", remove_unused_columns=False
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [[label_list[p] for p in pred] for pred in predictions]
    true_labels = [[label_list[l] for l in label] for label in labels]
    return classification_report(true_labels, true_predictions, output_dict=True)

In [10]:
# Split dataset into train and validation
tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.1)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    push_to_hub=False,
    report_to="none"  # Disable reporting to wandb/huggingface
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.argmax(torch.tensor(predictions), dim=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
    }

In [16]:
from transformers import TrainerCallback

# Define a callback function to save checkpoints
class SaveModelCheckpoint(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        output_dir = './checkpoint'
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        print(f"Checkpoint saved at epoch {state.epoch}")

# Add the callback to the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[SaveModelCheckpoint]  # Use the custom callback for saving checkpoints
)


<ipython-input-16-f00a8064d4eb>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
print(tokenized_datasets["train"][0])

{'text': ', O', 'tokens': [','], 'ner_tags': [0], 'input_ids': [0, 6, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the model and tokenizer from the checkpoint
checkpoint_dir = './checkpoint'  # Replace with the actual directory where your checkpoint was saved

model = AutoModelForTokenClassification.from_pretrained(checkpoint_dir)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

# Now, you can resume training or use the model for evaluation or inference


In [ ]:
from transformers import Trainer, TrainingArguments

# Create training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Output directory
    num_train_epochs=3,      # Number of epochs you want to train
    per_device_train_batch_size=16,  # Batch size
    per_device_eval_batch_size=64,   # Evaluation batch size
    evaluation_strategy="epoch",     # When to evaluate during training
    save_strategy="epoch",  # Save model checkpoints every epoch
    logging_dir="./logs",    # Directory for logs
)

# If you want to continue training with the previous state
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Resume training from checkpoint (if saved during previous training)
trainer.train(resume_from_checkpoint=True)
